In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import time
# yelp api stuff
from yelpapi import YelpAPI
# web scraping
import requests
from bs4 import BeautifulSoup
# for inspection dates
# import datetime
import pickle as pkl

In [2]:
# map stuff
import folium
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

In [3]:
%matplotlib inline

In [4]:
main_directory = r'C:\Users\ASUS\Desktop\ML-data science\DataIncubator\yelp-cleaners' + '\\'
data_directory = main_directory + 'data' + '\\'
# raw data
inspection_data_directory = data_directory + r'Inspection data\Austin' + '\\'
yelp_data_directory = main_directory + r'\data\yelp data\austin dataset' + '\\'
# files I generate here
processed_data_directory = main_directory + r'data\processed data' + '\\'

# old stuff
map_path =  main_directory + r'data\states_21basic\states.shp'

In [5]:
# get yelp api key stored in txt file
f = open(data_directory + 'yelp_api_key.txt','r')
api_key = f.read()
f.close()

In [6]:
# not that big
df = pd.read_csv(inspection_data_directory + 'Food_Establishment_Inspection_Scores.csv',
                delimiter = ',', on_bad_lines='skip')

# clean up address data, extracting lat/long
# formatted as Address \n city, state, zip \n (lat, long)
df.Address = df.Address.apply(lambda x: x.split('\n'))
df['latitude'] = df.Address.apply(lambda x: eval(x[2])[0] if len(x) == 3 else None)
df['longitude'] = df.Address.apply(lambda x: eval(x[2])[1] if len(x) == 3 else None)
df['Address Line 2'] = df.Address.str[1]
df.Address = df.Address.str[0]
df['Inspection Date'] = pd.to_datetime(df['Inspection Date'])

In [7]:
df

,Restaurant Name,Zip Code,Inspection Date,Score,Address,Facility ID,Process Description,latitude,longitude,Address Line 2
0,LW - Crown Donut,78734.0,2019-12-06,97,325 S FM 620 RD,10545024,Routine Inspection,NaN,NaN,"LAKEWAY, TX 78734"
1,Riverside Grocery,78701.0,2022-03-03,99,30 N IH,2803648,Routine Inspection,NaN,NaN,"AUSTIN, TX 78701"
2,Circle K #2741303,78748.0,2021-06-25,91,8628 MANCHACA RD,2803589,Routine Inspection,NaN,NaN,"AUSTIN, TX 78748"
3,Stop 71,78735.0,2020-03-03,78,8314 W SH,2803909,Routine Inspection,NaN,NaN,"AUSTIN, TX 78735"
4,Angel's Ice House,78669.0,2021-11-16,94,21815 W SH,2802178,Routine Inspection,NaN,NaN,"SPICEWOOD, TX 78669"
...,...,...,...,...,...,...,...,...,...,...
25779,"Commodore Perry Estate, The",78751.0,2021-11-18,84,4114 RED RIVER ST,12392635,Routine Inspection,30.300755,-97.721903,"AUSTIN, TX 78751"
25780,El Mercado North,78757.0,2021-07-27,89,7414 BURNET RD,2800466,Routine Inspection,30.350003,-97.734632,"AUSTIN, TX 78757"
25781,Wendy's #W129/11274,78759.0,2019-12-31,82,10701 RESEARCH BLVD,11115872,Routine Inspection,30.398568,-97.745647,"AUSTIN, TX 78759"
25782,Slapbox Pizzicheria,78717.0,2021-10-18,80,9900 W PARMER LN,12393453,Routine Inspection,30.486294,-97.770553,"AUSTIN, TX 78717"
